In [2]:
pip install pycryptodome cryptography


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 41.3 MB/s eta 0:00:00


In [7]:
import os
import time
import hashlib
from Crypto.Cipher import AES, PKCS1_OAEP
from Crypto.PublicKey import RSA
from Crypto.Random import get_random_bytes
from Crypto.Signature import pkcs1_15
from Crypto.Hash import SHA256
import matplotlib.pyplot as plt

# -------------------------
# Key Generation Functions
# -------------------------
def generate_aes_key(key_size):
    key_file = f"aes_{key_size}.key"
    if not os.path.exists(key_file):
        key = get_random_bytes(key_size // 8)
        with open(key_file, "wb") as f:
            f.write(key)
    else:
        with open(key_file, "rb") as f:
            key = f.read()
    return key

def generate_rsa_keys():
    if not os.path.exists("rsa_private.pem") or not os.path.exists("rsa_public.pem"):
        key = RSA.generate(2048)
        private_key = key.export_key()
        public_key = key.publickey().export_key()
        with open("rsa_private.pem", "wb") as f:
            f.write(private_key)
        with open("rsa_public.pem", "wb") as f:
            f.write(public_key)
    else:
        with open("rsa_private.pem", "rb") as f:
            private_key = f.read()
        with open("rsa_public.pem", "rb") as f:
            public_key = f.read()
    return private_key, public_key

# -------------------------
# AES Encryption/Decryption
# -------------------------
def aes_encrypt_decrypt():
    key_size = int(input("Enter AES key size (128 or 256): "))
    mode_input = input("Enter mode (ECB or CFB): ").upper()
    key = generate_aes_key(key_size)

    data = input("Enter text to encrypt: ").encode()

    if mode_input == "ECB":
        cipher = AES.new(key, AES.MODE_ECB)
        # Pad to multiple of 16 bytes
        pad_len = 16 - len(data) % 16
        data += bytes([pad_len]) * pad_len
        encrypted = cipher.encrypt(data)
        decrypted = cipher.decrypt(encrypted)
        decrypted = decrypted[:-decrypted[-1]]  # remove padding
    elif mode_input == "CFB":
        iv = get_random_bytes(16)
        cipher = AES.new(key, AES.MODE_CFB, iv=iv)
        encrypted = cipher.encrypt(data)
        cipher_dec = AES.new(key, AES.MODE_CFB, iv=iv)
        decrypted = cipher_dec.decrypt(encrypted)
    else:
        print("Invalid mode")
        return

    print("Encrypted:", encrypted.hex())
    print("Decrypted:", decrypted.decode())

# -------------------------
# RSA Encryption/Decryption
# -------------------------
def rsa_encrypt_decrypt():
    private_key_data, public_key_data = generate_rsa_keys()
    private_key = RSA.import_key(private_key_data)
    public_key = RSA.import_key(public_key_data)

    data = input("Enter text to encrypt: ").encode()
    cipher_rsa = PKCS1_OAEP.new(public_key)
    encrypted = cipher_rsa.encrypt(data)

    cipher_rsa_dec = PKCS1_OAEP.new(private_key)
    decrypted = cipher_rsa_dec.decrypt(encrypted)
    print("Encrypted:", encrypted.hex())
    print("Decrypted:", decrypted.decode())

# -------------------------
# RSA Signature
# -------------------------
def rsa_sign_verify():
    private_key_data, public_key_data = generate_rsa_keys()
    private_key = RSA.import_key(private_key_data)
    public_key = RSA.import_key(public_key_data)

    data = input("Enter text to sign: ").encode()
    h = SHA256.new(data)
    signature = pkcs1_15.new(private_key).sign(h)

    print("Signature:", signature.hex())

    try:
        pkcs1_15.new(public_key).verify(h, signature)
        print("Signature verified successfully!")
    except (ValueError, TypeError):
        print("Signature verification failed.")

# -------------------------
# SHA-256 Hash
# -------------------------
def sha256_hash():
    data = input("Enter text to hash: ").encode()
    digest = hashlib.sha256(data).hexdigest()
    print("SHA-256 Hash:", digest)

# -------------------------
# Execution Time Measurement
# -------------------------
def measure_time(func, *args):
    start = time.time()
    func(*args)
    end = time.time()
    print(f"Elapsed time: {end-start:.6f} seconds")

# -------------------------
# Menu
# -------------------------
def menu():
    while True:
        print("\n--- Crypto Lab Menu ---")
        print("1. AES Encryption/Decryption")
        print("2. RSA Encryption/Decryption")
        print("3. RSA Signature")
        print("4. SHA-256 Hash")
        print("5. Exit")
        choice = input("Enter your choice: ")

        if choice == "1":
            measure_time(aes_encrypt_decrypt)
        elif choice == "2":
            measure_time(rsa_encrypt_decrypt)
        elif choice == "3":
            measure_time(rsa_sign_verify)
        elif choice == "4":
            measure_time(sha256_hash)
        elif choice == "5":
            break
        else:
            print("Invalid choice!")

if __name__ == "__main__":
    menu()



--- Crypto Lab Menu ---
1. AES Encryption/Decryption
2. RSA Encryption/Decryption
3. RSA Signature
4. SHA-256 Hash
5. Exit
Enter your choice: 1
Enter AES key size (128 or 256): 128
Enter mode (ECB or CFB): ECB
Enter text to encrypt: testing
Encrypted: 1f24b4286386f23251d365d75a1bb944
Decrypted: testing
Elapsed time: 16.358360 seconds

--- Crypto Lab Menu ---
1. AES Encryption/Decryption
2. RSA Encryption/Decryption
3. RSA Signature
4. SHA-256 Hash
5. Exit
Enter your choice: 2
Enter text to encrypt: testing
Encrypted: 1bf7bc3783ab25504131a291308e09c8b9e2d0e0973ce13354321772a600a063e0c2587f38e4e4deca903ad635d592255920d1a6887fb4673595ce0a0f7b8483c6029e14f6a14c5b1424254c22e3d6cb51d74ee7c26f9d8d13fde194ab67ad5a861a3c911a376d84cdead8a76607f8525913bfda16825352ca62f3efdbb94a3374b51f05b9584a33662d7d5e09f5fec3a44f31bf88a22aa7e4af55b1dc81489011944f3a9811249ba5b6f012ab0f6d93a5a765e0b2bebb2341bf474e72a52cc5e276510694dde8b5c19a596d484e1dc1056934057da8fb619bdeda36150b96972b7adc048b6970e3442dd0e61cfc